In [1]:
from sklearn.preprocessing import StandardScaler
from sail.models.torch.tcn import TCNRegressor
import numpy as np
from sklearn.datasets import make_regression

In [2]:
# Create data as we do in tests
def regression_data(n_samples, n_features):
    X, y = make_regression(  # type: ignore
        n_samples, n_features, n_informative=10, bias=0, random_state=0
    )
    X, y = X.astype(np.float32), y.astype(np.float32).reshape(-1, 1)
    Xt = StandardScaler().fit_transform(X)
    yt = StandardScaler().fit_transform(y)
    return Xt, yt


In [3]:
# Set size of data
n_samples = 1000
n_features = 12
batch_size = 20

# Get regression data
X, y = regression_data(n_samples, n_features)

# Add dummy axis, as net requires it
X = X[:, :, np.newaxis]

# Initialize network as TCNRegressor
learner_tcn = TCNRegressor(n_features, 1)

# Calculate epochs as even divisor of n_samples and batch_size for sake of easiness
epochs = n_samples // batch_size
for i in range(epochs):
    # Extract X and y values of batch_size from training set
    X_step = X[i * batch_size : (i + 1) * batch_size, :, :]
    y_step = y[i * batch_size : (i + 1) * batch_size]

    # Run partial fit
    learner_tcn.partial_fit(X_step, y_step)

# Run prediction (note: this is of course the data that we trained on,
# but only for convenience sake; we are just testing if partial_fit works)
y_prediction = learner_tcn.predict(X)


  epoch    train_loss     dur
-------  ------------  ------
      1        1.3681  0.0596
      2        1.1208  0.0057
      3        1.6188  0.0099
      4        1.8581  0.0084
      5        1.2468  0.0167
      6        0.7707  0.0169
      7        1.8506  0.0152
      8        1.5918  0.0105
      9        1.7303  0.0078
     10        1.4191  0.0063
     11        1.2712  0.0067
     12        1.0703  0.0095
     13        0.4689  0.0060
     14        1.6538  0.0095
     15        0.7638  0.0224
     16        0.9005  0.0118
     17        0.7562  0.0324
     18        0.9287  0.0304
     19        0.6940  0.0272
     20        1.7177  0.0341
     21        0.6680  0.0120
     22        0.9238  0.0063
     23        0.7282  0.0102
     24        1.9051  0.0182
     25        0.8007  0.0204
     26        0.7136  0.0105
     27        1.2823  0.0196
     28        0.7282  0.0097
     29        0.9495  0.0087
     30        1.0306  0.0104
     31        0.7679  0.0245
     32   

In [4]:
# Plot the predictions and the truth
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "vscode"

y_pred = y_prediction.flatten().tolist()
y_true = y.flatten().tolist()
x = np.arange(len(y_true))
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=x, y=y_pred, name="predicted", line=dict(color="red", width=2))
)
fig.add_trace(
    go.Scatter(x=x, y=y_true, name="actual", line=dict(color="blue", width=2))
)
fig.show()
